In [2]:
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.mip_solver import mip_solve
from cloudmanufacturing.validation import construct_delta, objvalue

In [3]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")
for problem in dataset:
    print(f'Problem: {problem["name"]}')

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:05<00:00,  3.48it/s]

Problem: 5,10,10-1
Problem: 5,10,10-2
Problem: 5,10,10-3
Problem: 10,10,10-1
Problem: 10,10,10-2
Problem: 10,10,10-3
Problem: 5,10,20-1
Problem: 5,10,20-2
Problem: 5,10,20-3
Problem: 5,20,10-1
Problem: 5,20,10-2
Problem: 5,20,10-3
Problem: 5,20,20-1
Problem: 5,20,20-2
Problem: 5,20,20-3
Problem: 5,5,5-1
Problem: 5,5,5-2
Problem: 5,5,5-3


In [4]:
problem = dataset[0]
delta, gamma, status, value = mip_solve(problem)
print(f'Problem {problem["name"]}: {value:.2f}, {status}')

Problem 5,10,10-1: 5086.08, OptimizationStatus.OPTIMAL


In [5]:
print(problem["time_cost"].round(2))

[[99.    3.79  6.28  6.53  5.19  4.38  6.76 99.    4.76  3.38]
 [99.   99.    3.18  3.16  4.91  6.4   4.28  4.27 99.    3.27]
 [ 3.63 99.   99.    4.38  6.83  6.28  5.53 99.    5.93  5.65]
 [99.    5.43 99.    3.23  6.98  3.81  6.5   4.22  5.75  6.9 ]
 [ 6.16 99.    6.39  3.49  3.93  3.59 99.   99.   99.   99.  ]
 [ 3.49  3.71  6.79 99.    5.45  5.49 99.    4.75  4.43  3.65]
 [ 4.39  5.11  6.72  6.47  5.23 99.    5.74  3.98  6.79  5.84]
 [ 5.73 99.    4.96  4.59  6.23  4.7   3.69  4.26  6.77  5.35]
 [99.    6.96  6.28 99.    6.55  5.93  3.75  6.08  4.9   3.06]
 [99.   99.    3.86  3.17  6.77  4.12  4.29  5.37  5.84  4.69]]


In [6]:
objvalue(problem, gamma, delta)

5086.083142240974

In [7]:
objvalue(problem, gamma, construct_delta(problem, gamma))

5086.083142240974

In [19]:
import numpy as np

In [59]:
def find_path(operation):
    data = []
    path = []
    for i in range(len(gamma)):
        sub_operation = gamma[i][operation]
        if len(np.nonzero(sub_operation)[0] > 0):
            path.append(np.nonzero(sub_operation)[0][0])
        data.append(sub_operation.tolist())
    return path, data

In [70]:
path,data = find_path(1)
path

[1, 1, 1, 1]